<a href="https://colab.research.google.com/github/theantigone/ngram-java-ai/blob/main/NGRAM_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from pygments.lexers.jvm import JavaLexer
from pygments.token import Token
from collections import defaultdict, Counter
import math
import json
import random

def tokenize_java_code(code):
    """Tokenizes Java code using Pygments."""
    lexer = JavaLexer()
    return [t[1] for t in lexer.get_tokens(code) if t[0] not in Token.Text]


In [4]:
def build_ngram_model(corpus, n):
    """Builds an N-gram model from tokenized Java methods."""
    ngram_counts = defaultdict(int)
    context_counts = defaultdict(int)

    for method in corpus:
        tokens = tokenize_java_code(method)
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i:i+n])
            context = tuple(tokens[i:i+n-1])
            ngram_counts[ngram] += 1
            context_counts[context] += 1

    return ngram_counts, context_counts


In [5]:
def compute_probabilities(ngram_counts, context_counts):
    """Computes conditional probabilities for N-grams."""
    probabilities = {}
    for ngram, count in ngram_counts.items():
        context = ngram[:-1]
        probabilities[ngram] = count / context_counts[context]
    return probabilities


In [6]:
def perplexity(test_corpus, probabilities, n):
    """Computes perplexity for the N-gram model on a test set."""
    log_prob_sum = 0
    N = 0

    for method in test_corpus:
        tokens = tokenize_java_code(method)
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i:i+n])
            prob = probabilities.get(ngram, 1e-6)  # Smoothing for unseen cases
            log_prob_sum += math.log2(prob)
            N += 1

    return 2 ** (-log_prob_sum / N)


In [7]:
def iterative_prediction_x(probabilities, context, n):
    """Predicts the next token iteratively given a starting context."""
    predictions = []

    for _ in range(10):  # Limit predictions to avoid infinite loops
        candidates = {ngram[-1]: prob for ngram, prob in probabilities.items() if ngram[:-1] == context}

        if not candidates:
            break  # Stop if no valid prediction

        next_token = max(candidates, key=candidates.get)  # Most probable token
        predictions.append((next_token, round(candidates[next_token], 3)))
        context = context[1:] + (next_token,)  # Update context

    return predictions


In [8]:
def generate_results_json(test_corpus, probabilities, n, filename):
    """Generates a JSON file with token predictions and probabilities."""
    results = {}

    for i, method in enumerate(test_corpus[:100]):  # Limit to 100 examples
        tokens = tokenize_java_code(method)[:n-1]
        context = tuple(tokens)
        results[str(i)] = iterative_prediction_x(probabilities, context, n)

    with open(filename, "w") as f:
        json.dump(results, f, indent=4)

    print(f"Saved predictions to {filename}")


In [9]:
def main():
    # Load dataset
    with open("/content/drive/Shareddrives/CSCI_420/data/new.txt", "r") as f:
        corpus = [line.strip() for line in f]

    # Shuffle and split dataset
    random.shuffle(corpus)
    train_set = corpus[:int(0.8 * len(corpus))]
    eval_set = corpus[int(0.8 * len(corpus)):int(0.9 * len(corpus))]
    test_set = corpus[int(0.9 * len(corpus)):]

    # Train and evaluate for different N values
    best_n = None
    best_perplexity = float("inf")
    results = {}

    for n in [3, 5, 7]:
        ngram_counts, context_counts = build_ngram_model(train_set, n)
        probabilities = compute_probabilities(ngram_counts, context_counts)

        pp = perplexity(eval_set, probabilities, n)
        results[n] = pp

        if pp < best_perplexity:
            best_perplexity = pp
            best_n = n

    print(f"Best N: {best_n}, Perplexity: {best_perplexity}")

    # Train best model
    best_ngram_counts, best_context_counts = build_ngram_model(train_set, best_n)
    best_probabilities = compute_probabilities(best_ngram_counts, best_context_counts)

    # Generate JSON output for student model
    generate_results_json(test_set, best_probabilities, best_n, "results_student_model.json")

    # Train model on instructor's dataset
    with open("/content/drive/Shareddrives/CSCI_420/data/training.txt", "r") as f:
        instructor_corpus = [line.strip() for line in f]

    random.shuffle(instructor_corpus)
    instructor_train_set = instructor_corpus[:int(0.8 * len(instructor_corpus))]
    instructor_eval_set = instructor_corpus[int(0.8 * len(instructor_corpus)):int(0.9 * len(instructor_corpus))]

    # Find best N for instructor's dataset
    best_instructor_n = None
    best_instructor_perplexity = float("inf")

    for n in [3, 5, 7]:
        ngram_counts, context_counts = build_ngram_model(instructor_train_set, n)
        probabilities = compute_probabilities(ngram_counts, context_counts)

        pp = perplexity(instructor_eval_set, probabilities, n)
        if pp < best_instructor_perplexity:
            best_instructor_perplexity = pp
            best_instructor_n = n

    print(f"Best N for instructor model: {best_instructor_n}, Perplexity: {best_instructor_perplexity}")

    # Train best instructor model
    best_instructor_ngram_counts, best_instructor_context_counts = build_ngram_model(instructor_train_set, best_instructor_n)
    best_instructor_probabilities = compute_probabilities(best_instructor_ngram_counts, best_instructor_context_counts)

    # Generate JSON output for instructor model
    generate_results_json(test_set, best_instructor_probabilities, best_instructor_n, "results_teacher_model.json")

if __name__ == "__main__":
    main()


Best N: 3, Perplexity: 34.38444856185524
Saved predictions to results_student_model.json
Best N for instructor model: 3, Perplexity: 26.398874090998675
Saved predictions to results_teacher_model.json


In [14]:
# Train on student dataset only
def main():
    # Load dataset
    with open("/content/drive/Shareddrives/CSCI_420/data/new.txt", "r") as f:
        corpus = [line.strip() for line in f]

    # Shuffle and split dataset
    random.shuffle(corpus)
    train_set = corpus[:int(0.8 * len(corpus))]
    eval_set = corpus[int(0.8 * len(corpus)):int(0.9 * len(corpus))]
    test_set = corpus[int(0.9 * len(corpus)):]

    # Train and evaluate for different N values
    best_n = None
    best_perplexity = float("inf")
    results = {}

    for n in [3, 5, 7]:
        ngram_counts, context_counts = build_ngram_model(train_set, n)
        probabilities = compute_probabilities(ngram_counts, context_counts)

        pp = perplexity(eval_set, probabilities, n)
        results[n] = pp

        if pp < best_perplexity:
            best_perplexity = pp
            best_n = n

    print(f"Best N: {best_n}, Perplexity: {best_perplexity}")

    # Train best model
    best_ngram_counts, best_context_counts = build_ngram_model(train_set, best_n)
    best_probabilities = compute_probabilities(best_ngram_counts, best_context_counts)

    # Generate JSON output for student model
    generate_results_json(test_set, best_probabilities, best_n, "results_student_model.json")

    # # Train model on instructor's dataset
    # with open("/content/drive/Shareddrives/CSCI_420/data/training.txt", "r") as f:
    #     instructor_corpus = [line.strip() for line in f]

    # random.shuffle(instructor_corpus)
    # instructor_train_set = instructor_corpus[:int(0.8 * len(instructor_corpus))]
    # instructor_eval_set = instructor_corpus[int(0.8 * len(instructor_corpus)):int(0.9 * len(instructor_corpus))]

    # # Find best N for instructor's dataset
    # best_instructor_n = None
    # best_instructor_perplexity = float("inf")

    # for n in [3, 5, 7]:
    #     ngram_counts, context_counts = build_ngram_model(instructor_train_set, n)
    #     probabilities = compute_probabilities(ngram_counts, context_counts)

    #     pp = perplexity(instructor_eval_set, probabilities, n)
    #     if pp < best_instructor_perplexity:
    #         best_instructor_perplexity = pp
    #         best_instructor_n = n

    # print(f"Best N for instructor model: {best_instructor_n}, Perplexity: {best_instructor_perplexity}")

    # # Train best instructor model
    # best_instructor_ngram_counts, best_instructor_context_counts = build_ngram_model(instructor_train_set, best_instructor_n)
    # best_instructor_probabilities = compute_probabilities(best_instructor_ngram_counts, best_instructor_context_counts)

    # # Generate JSON output for instructor model
    # generate_results_json(test_set, best_instructor_probabilities, best_instructor_n, "results_teacher_model.json")

if __name__ == "__main__":
    main()


Best N: 3, Perplexity: 84.12493880450842
Saved predictions to results_student_model.json


In [15]:
from google.colab import files
files.download('results_student_model.json')
files.download('results_teacher_model.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>